# Experiments 16-18 (640 px)

- **Model:**
    1. `yolov8n` *(Nano)*
    1. `yolov8m` *(Medium)*
- **Dataset:** 3.5m | 90º
- **Crop Tiles:** 640px
- **Sizes:** small & mid
- **Experiments:**
    1. 100 epochs *(Nano)*
    2. 100 epochs *(Medium)*
    3. 1000 epochs *(Nano)*

Previous experiments have shown that:
1. GPU resources weren't being used much during training.
2. YOLO model was giving the following warning:

    `WARNING ⚠️ imgsz=[240] must be multiple of max stride 32, updating to [256]`

Therefore, for these next experiments, we decided to crop images into bigger tiles of **640px**.

## Init

In [ ]:
import os
import shutil
import fnmatch
import pickle

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.8/921.8 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

## Helper Functions

In [3]:
# Change for different file formats
reference = {
  "small": {
    "suffix": ".S",
    "file": "209"
  },
  "mid": {
    "suffix": ".M",
    "file": "503"
  },
  "large": {
    "suffix": ".L",
    "file": "000"
  }
}

In [4]:
# Clone config files
def copy_config(src_folder, dest_folder):
    """
    Copies files from src_folder to dest_folder, excluding subfolders.

    Args:
        src_folder: The path to the source folder.
        dest_folder: The path to the destination folder.
    """

    try:
        # Ensure destination folder exists
        os.makedirs(dest_folder, exist_ok=True)

        for filename in os.listdir(src_folder):
            src_path = os.path.join(src_folder, filename)
            dest_path = os.path.join(dest_folder, filename)

            if os.path.isfile(src_path):
                shutil.copy2(src_path, dest_path) #copy metadata as well.
                #Use shutil.copy for not copying metadata.
                print(f"Copied: {filename}")
            #else: #optional
                #print(f"Skipped (not a file): {filename}") #optional. Uncomment if you want to see the skipped folders.

        print("✅ Copying complete.")

    except Exception as e:
        print(f"❌ An error occurred: {e}")


In [5]:
# Copy filtered dataset images/labels
def copy_and_filter_folder(src_folder, dest_folder, pattern):
    """
    Copies a folder and files that match the given pattern.
    Alerts the user when a folder or file already exists but *does not* overwrite.
    Creates only what is needed.

    :param src_folder: Path to the source folder.
    :param dest_folder: Path to the destination folder.
    :param pattern: Filename pattern to keep (e.g., "*.txt").
    """
    try:
        # Ensure destination folder exists
        if not os.path.exists(dest_folder):
            print(f"✓ Creating destination folder '{dest_folder}'.\n")
            os.makedirs(dest_folder)
        else:
            print(f"✓ Destination folder '{dest_folder}' already exists.\n")

        # Walk through the source folder
        for root, _, files in os.walk(src_folder):
            relative_path = os.path.relpath(root, src_folder)
            new_root = os.path.join(dest_folder, relative_path)

            if not os.path.exists(new_root):
                print(f"Creating subdirectory '{new_root}'")
                os.makedirs(new_root)
            else:
                print(f"❕Subdirectory '{new_root}' already exists.")
                print("Make sure the data inside is relevant. Otherwise, just delete the folder and repeat the cloning process.")

            for file in files:
                if fnmatch.fnmatch(file, pattern + "*"):
                    src_file = os.path.join(root, file)
                    dest_file = os.path.join(new_root, file)

                    if not os.path.exists(dest_file):
                        shutil.copy2(src_file, dest_file)  # copy metadata as well
                    else:
                        print(f"❗️File '{dest_file}' already exists. Skipping.")

            print(f" ✓ Copying files complete.\n")
        print("✅ Copying dataset complete.")

    except Exception as e:
        print(f"❌ An error occurred: {e}")

In [6]:
def copy_directory(source, destination, overwrite=True):
    try:
        if overwrite and os.path.exists(destination):
            shutil.rmtree(destination)  # Remove existing directory
        shutil.copytree(source, destination)
        print(f"Directory '{source}' copied to '{destination}'")
        return True  # Indicate success
    except FileExistsError:
        print(f"Destination '{destination}' exists. Use overwrite=True to replace.")
        return False  # Indicate failure

In [ ]:
# Load/Download prediction results (BB + confidence) as .pkl file

def save_results(results, filename):
    with open(filename, 'wb') as f:
        pickle.dump(results, f)

def load_results(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [7]:
def save_on_cloud(source: str, destination: str):
    """
    Saves a folder to a cloud storage location (e.g., Google Drive in Colab).

    Args:
        source (str): The path to the source folder.
        destination (str): The path to the destination folder (in cloud storage).
    """
    # 0. Input Validation (Assertions)
    assert isinstance(source, str), "Source must be a string."
    assert isinstance(destination, str), "Destination must be a string."

    try:
        # 1. Verify Source Folder
        if not os.path.exists(destination):
            os.makedirs(destination)

        # 2. Copy the Folder
        shutil.copytree(source, destination, dirs_exist_ok=True)
        print("✅ Folder copied successfully:\n  ",source,"\n  -->",destination)

    except Exception as e:
        print(f"❌ An error occurred: {e}")

# Datasets builder

## Importing from Drive

In [8]:
!rm -rf /content/sample_data

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Check if the cloud path is ok and the dataset can be found
!ls /content/drive/MyDrive/YOLO

3.5m.v3i.yolov8  3.5m.v3i.yolov8.640px	3m60.v2i.yolov8  5m60.v1i.yolov8  5m.v1i.yolov8


In [10]:
drive_path = '/content/drive/MyDrive/YOLO'
drive_datasets_paths = os.listdir(drive_path)
drive_datasets = len(drive_datasets_paths)
if (drive_datasets) > 1:
    print("There are %d dataset options:" % drive_datasets)
else:
    print("Theres is only 1 dataset:")
drive_datasets_paths

There are 5 dataset options:


['3.5m.v3i.yolov8',
 '5m.v1i.yolov8',
 '5m60.v1i.yolov8',
 '3m60.v2i.yolov8',
 '3.5m.v3i.yolov8.640px']

In [ ]:
choose_dataset = 5
index = choose_dataset - 1
model = os.listdir(drive_path)[index]
print("Chosen model:", model)

Choosed model: 3.5m.v3i.yolov8.640px


***Readme:***
*   **Option 1:** is desirable if you need to test many subset combinations in the same session (avoid downloading data twice from the cloud).
*   **Option 2:** is desired if you are goint to work just with one dataset  (avoid downloading unnecessary data from the cloud).
*   **Option 3:** is best if you're just going to test one subset combination  (avoid downloading any data from the cloud).



In [ ]:
# Option 1 (download all datasets)
!cp -r /content/drive/MyDrive/YOLO/ /content
src_folder = f"/content/YOLO/{model}"

In [20]:
# Option 2 (download just the needed dataset)
cloud_path = f"/content/drive/MyDrive/YOLO/{model}/"
local_path = f"/content/YOLO/"
!mkdir $local_path
!cp -r $cloud_path $local_path
src_folder = f"/content/YOLO/{model}"

In [ ]:
# Option 3 (don't download it, just open it from the cloud)
src_folder = f"{drive_path}/{model}"

# Model constructor
1. Model 1: `yolov8n` *(Nano)*
2. Model 2: `yolov8m` *(Medium)*

## Download model

In [21]:
from ultralytics import YOLO

# Load the YOLO model 1
model = YOLO("yolov8n.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 320MB/s]


In [22]:
# Load the YOLO model 2
modelM = YOLO("yolov8m.pt")

100%|██████████| 49.7M/49.7M [00:01<00:00, 49.2MB/s]


# Finetuning

### Info

In [ ]:
!nvidia-smi

Tue Feb 25 16:01:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!yolo version

8.3.79


-----
## Experiment 16
### *Nano | 100 epochs*

### Train

In [ ]:
# Train Nano model
model.train(
    data="/content/YOLO/3.5m.v3i.yolov8.640px/data.yaml",
    epochs=100,
    imgsz=640,
    freeze=10,
    batch=64
)

Ultralytics 8.3.79 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/YOLO/3.5m.v3i.yolov8.640px/data.yaml, epochs=100, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

100%|██████████| 755k/755k [00:00<00:00, 52.8MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 207MB/s]


AMP: checks passed ✅


...

                   all        108       2409      0.486       0.42      0.418      0.143



100 epochs completed in 0.143 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.3MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.79 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.22s/it]


                   all        108       2409      0.485       0.42      0.417      0.143
Speed: 0.2ms preprocess, 2.2ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78292d09b390>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Validation

In [ ]:
# Load currently trained YOLO model
model1 = YOLO("/content/runs/detect/train/weights/best.pt")

In [ ]:
# Load stored model
# model1 = YOLO("/content/drive/MyDrive/save/detect/train/weights/best.pt")

In [40]:
# Validate the model
model1.val(data="/content/YOLO/3.5m.v3i.yolov8.640px/data.yaml")

Ultralytics 8.3.79 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/YOLO/3.5m.v3i.yolov8.640px/valid/labels.cache... 108 images, 0 backgrounds, 0 corrupt: 100%|██████████| 108/108 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:05<00:00,  1.29it/s]


                   all        108       2409      0.484      0.419      0.416      0.143
Speed: 4.4ms preprocess, 6.4ms inference, 0.0ms loss, 3.5ms postprocess per image
Results saved to runs/detect/val2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7efb62cd69d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Inference

#### Test 1 (small weed)

In [41]:
image = '/content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/209_205_50_JPG.tile01x01.rf.949b5aa6fdbf6a1f721c39ed5ddcebec.jpg'
keyword = '/images/'
file_name = image[image.find(keyword)+len(keyword):]

In [42]:
project="model1(Nano)"
experiment="experiment1"

In [43]:
# Ejecutar predicción sobre la imagen
results = model1(image, save=True, project=project, name=experiment)

# Acceder a las coordenadas de los bounding boxes
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas del bounding box (xmin, ymin, xmax, ymax)
        conf = box.conf[0]  # Confianza de la predicción
        cls = box.cls[0]  # Clase predicha
        print(f"Clase: {int(cls)}, Confianza: {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")


image 1/1 /content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/209_205_50_JPG.tile01x01.rf.949b5aa6fdbf6a1f721c39ed5ddcebec.jpg: 640x640 4 plant-weeds, 7.9ms
Speed: 3.8ms preprocess, 7.9ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to model1(Nano)/experiment1
Clase: 0, Confianza: 0.41, BBox: (178.24, 188.32, 236.01, 213.56)
Clase: 0, Confianza: 0.39, BBox: (0.15, 162.20, 38.97, 181.10)
Clase: 0, Confianza: 0.27, BBox: (86.10, 184.91, 130.08, 209.06)
Clase: 0, Confianza: 0.27, BBox: (49.90, 169.50, 104.35, 196.66)


In [44]:
filename = f"/content/{project}/{experiment}/results.pkl"
save_results(results, filename)

#### Test 2 (medium weed)

In [45]:
image = '/content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/503_118_29_JPG.tile01x01.rf.33b1e23ce2d5d55c753d42f7fae09523.jpg'
keyword = '/images/'
file_name = image[image.find(keyword)+len(keyword):]

In [46]:
project="model1(Nano)"
experiment="experiment2"

In [47]:
# Ejecutar predicción sobre la imagen
results = model1(image, save=True, project=project, name=experiment)

# Acceder a las coordenadas de los bounding boxes
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas del bounding box (xmin, ymin, xmax, ymax)
        conf = box.conf[0]  # Confianza de la predicción
        cls = box.cls[0]  # Clase predicha
        print(f"Clase: {int(cls)}, Confianza: {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")


image 1/1 /content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/503_118_29_JPG.tile01x01.rf.33b1e23ce2d5d55c753d42f7fae09523.jpg: 640x640 5 plant-weeds, 7.2ms
Speed: 2.6ms preprocess, 7.2ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to model1(Nano)/experiment2
Clase: 0, Confianza: 0.57, BBox: (10.33, 1.58, 50.00, 50.03)
Clase: 0, Confianza: 0.53, BBox: (0.00, 27.18, 27.05, 73.38)
Clase: 0, Confianza: 0.39, BBox: (0.00, 156.45, 34.07, 194.97)
Clase: 0, Confianza: 0.35, BBox: (28.93, 2.01, 60.48, 40.61)
Clase: 0, Confianza: 0.29, BBox: (0.44, 206.66, 21.41, 240.00)


In [48]:
filename = f"/content/{project}/{experiment}/results.pkl"
save_results(results, filename)

### Save results

In [49]:
# Store model weights and metrics
save_on_cloud(source='/content/runs/', destination='/content/drive/MyDrive/save/')

✅ Folder copied successfully:
   /content/runs/ 
  --> /content/drive/MyDrive/save/


In [64]:
# Store experiments
save_on_cloud(source='/content/model1(Nano)/', destination='/content/drive/MyDrive/save.Nano/')

✅ Folder copied successfully:
   /content/model1(Nano)/ 
  --> /content/drive/MyDrive/save.Nano/


----------
## Experiment 17
### *Medium | 100 epochs*

### Train

In [ ]:
# Train Medium model
modelM.train(
    data="/content/YOLO/3.5m.v3i.yolov8.640px/data.yaml",
    epochs=100,
    imgsz=640,
    freeze=10,
    batch=64
)

Ultralytics 8.3.79 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/YOLO/3.5m.v3i.yolov8.640px/data.yaml, epochs=100, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

train: Scanning /content/YOLO/train/labels.cache... 216 images, 0 backgrounds, 0 corrupt: 100%|██████████| 216/216 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/YOLO/valid/labels.cache... 108 images, 0 backgrounds, 0 corrupt: 100%|██████████| 108/108 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        108       2409      0.536      0.484      0.476      0.169



100 epochs completed in 0.231 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 52.0MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.79 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.77s/it]


                   all        108       2409      0.536      0.484      0.476      0.169
Speed: 0.2ms preprocess, 10.0ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78292d09b990>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Validation

In [ ]:
# Load currently trained YOLO model
model2 = YOLO("/content/runs/detect/train2/weights/best.pt")

In [ ]:
# Load stored model
# model2 = YOLO("/content/drive/MyDrive/save2/detect/train2/weights/best.pt")

In [52]:
# Validate the model
model2.val(data="/content/YOLO/3.5m.v3i.yolov8.640px/data.yaml")

Ultralytics 8.3.79 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /content/YOLO/3.5m.v3i.yolov8.640px/valid/labels.cache... 108 images, 0 backgrounds, 0 corrupt: 100%|██████████| 108/108 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:07<00:00,  1.07s/it]


                   all        108       2409      0.531      0.489      0.477      0.168
Speed: 6.2ms preprocess, 22.3ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/val2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7efb6b65be10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Inference

#### Test 1 (small weed)

In [53]:
image = '/content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/209_205_50_JPG.tile01x01.rf.949b5aa6fdbf6a1f721c39ed5ddcebec.jpg'
keyword = '/images/'
file_name = image[image.find(keyword)+len(keyword):]

In [54]:
project="model2(Med)"
experiment="experiment1"

In [55]:
# Ejecutar predicción sobre la imagen
results = model2(image, save=True, project=project, name=experiment)

# Acceder a las coordenadas de los bounding boxes
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas del bounding box (xmin, ymin, xmax, ymax)
        conf = box.conf[0]  # Confianza de la predicción
        cls = box.cls[0]  # Clase predicha
        print(f"Clase: {int(cls)}, Confianza: {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")


image 1/1 /content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/209_205_50_JPG.tile01x01.rf.949b5aa6fdbf6a1f721c39ed5ddcebec.jpg: 640x640 8 plant-weeds, 36.2ms
Speed: 4.2ms preprocess, 36.2ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to model2(Med)/experiment1
Clase: 0, Confianza: 0.57, BBox: (0.00, 164.27, 34.81, 179.41)
Clase: 0, Confianza: 0.42, BBox: (52.51, 149.56, 89.15, 165.94)
Clase: 0, Confianza: 0.41, BBox: (123.45, 200.19, 163.34, 230.33)
Clase: 0, Confianza: 0.37, BBox: (179.92, 189.67, 238.31, 214.76)
Clase: 0, Confianza: 0.34, BBox: (54.35, 182.28, 87.89, 199.30)
Clase: 0, Confianza: 0.33, BBox: (124.64, 201.98, 181.60, 230.63)
Clase: 0, Confianza: 0.32, BBox: (191.42, 194.56, 227.60, 217.62)
Clase: 0, Confianza: 0.29, BBox: (139.78, 207.44, 181.94, 218.66)


In [56]:
filename = f"/content/{project}/{experiment}/results.pkl"
save_results(results, filename)

#### Test 2 (medium weed)

In [57]:
image = '/content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/503_118_29_JPG.tile01x01.rf.33b1e23ce2d5d55c753d42f7fae09523.jpg'
keyword = '/images/'
file_name = image[image.find(keyword)+len(keyword):]

In [58]:
project="model2(Med)"
experiment="experiment2"

In [59]:
# Ejecutar predicción sobre la imagen
results = model2(image, save=True, project=project, name=experiment)

# Acceder a las coordenadas de los bounding boxes
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas del bounding box (xmin, ymin, xmax, ymax)
        conf = box.conf[0]  # Confianza de la predicción
        cls = box.cls[0]  # Clase predicha
        print(f"Clase: {int(cls)}, Confianza: {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")


image 1/1 /content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/503_118_29_JPG.tile01x01.rf.33b1e23ce2d5d55c753d42f7fae09523.jpg: 640x640 7 plant-weeds, 36.3ms
Speed: 3.6ms preprocess, 36.3ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to model2(Med)/experiment2
Clase: 0, Confianza: 0.50, BBox: (0.16, 210.20, 17.57, 239.88)
Clase: 0, Confianza: 0.37, BBox: (9.49, 0.76, 51.19, 53.00)
Clase: 0, Confianza: 0.34, BBox: (35.10, 7.11, 58.48, 35.83)
Clase: 0, Confianza: 0.29, BBox: (20.19, 132.29, 84.62, 144.77)
Clase: 0, Confianza: 0.28, BBox: (0.00, 157.67, 21.27, 185.98)
Clase: 0, Confianza: 0.26, BBox: (0.09, 39.50, 29.95, 74.09)
Clase: 0, Confianza: 0.26, BBox: (0.03, 202.64, 24.32, 240.00)


In [60]:
filename = f"/content/{project}/{experiment}/results.pkl"
save_results(results, filename)

### Save results

In [61]:
# Store model weights and metrics
save_on_cloud(source='/content/runs/', destination='/content/drive/MyDrive/save2/')

✅ Folder copied successfully:
   /content/runs/ 
  --> /content/drive/MyDrive/save2/


In [65]:
# Store experiments
save_on_cloud(source='/content/model2(Med)/', destination='/content/drive/MyDrive/save.Med/')

✅ Folder copied successfully:
   /content/model2(Med)/ 
  --> /content/drive/MyDrive/save.Med/


-----
## Experiment 18
### *Nano | ~1000 epochs*

### Train

In [23]:
# Train Nano model
model.train(
    data="/content/YOLO/3.5m.v3i.yolov8.640px/data.yaml",
    epochs=1000,
    imgsz=640,
    freeze=10,
    batch=64,
    patience=50 # If the model does not improve for 50 epochs, training will be interrupted
)

Ultralytics 8.3.79 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/YOLO/3.5m.v3i.yolov8.640px/data.yaml, epochs=1000, time=None, patience=50, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_con

100%|██████████| 755k/755k [00:00<00:00, 126MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 349MB/s]


AMP: checks passed ✅


...

                   all        108       2409      0.438      0.456      0.397      0.133
EarlyStopping: Training stopped early as no improvement observed in last 50 epochs. Best results observed at epoch 176, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



226 epochs completed in 0.318 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.79 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.27s/it]


                   all        108       2409       0.46      0.472      0.418      0.141
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7efb6b4fd450>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Validation

In [25]:
# Load the trained YOLO model
model3 = YOLO("/content/runs/detect/train3/weights/best.pt")

# Validate the model
model3.val(data="/content/YOLO/3.5m.v3i.yolov8.640px/data.yaml")

Ultralytics 8.3.79 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/YOLO/3.5m.v3i.yolov8.640px/valid/labels.cache... 108 images, 0 backgrounds, 0 corrupt: 100%|██████████| 108/108 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:07<00:00,  1.03s/it]


                   all        108       2409      0.459      0.475      0.419      0.141
Speed: 2.6ms preprocess, 8.1ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to runs/detect/val


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7efb6f8eac50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

### Inference

#### Test 1 (small weed)

In [26]:
image = '/content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/209_205_50_JPG.tile01x01.rf.949b5aa6fdbf6a1f721c39ed5ddcebec.jpg'
keyword = '/images/'
file_name = image[image.find(keyword)+len(keyword):]

In [27]:
project="model3(Epochs)"
experiment="experiment1"

In [28]:
# Ejecutar predicción sobre la imagen
results = model3(image, save=True, project=project, name=experiment)

# Acceder a las coordenadas de los bounding boxes
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas del bounding box (xmin, ymin, xmax, ymax)
        conf = box.conf[0]  # Confianza de la predicción
        cls = box.cls[0]  # Clase predicha
        print(f"Clase: {int(cls)}, Confianza: {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")


image 1/1 /content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/209_205_50_JPG.tile01x01.rf.949b5aa6fdbf6a1f721c39ed5ddcebec.jpg: 640x640 15 plant-weeds, 7.3ms
Speed: 2.7ms preprocess, 7.3ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to model3(Epochs)/experiment1
Clase: 0, Confianza: 0.57, BBox: (52.11, 150.09, 90.55, 166.27)
Clase: 0, Confianza: 0.45, BBox: (122.48, 200.91, 162.82, 230.74)
Clase: 0, Confianza: 0.44, BBox: (122.31, 201.27, 150.33, 229.79)
Clase: 0, Confianza: 0.41, BBox: (81.86, 130.73, 105.47, 145.08)
Clase: 0, Confianza: 0.41, BBox: (178.55, 189.17, 234.35, 211.29)
Clase: 0, Confianza: 0.38, BBox: (0.12, 163.37, 48.78, 180.88)
Clase: 0, Confianza: 0.38, BBox: (210.54, 19.54, 224.93, 33.88)
Clase: 0, Confianza: 0.35, BBox: (60.88, 98.84, 90.03, 108.90)
Clase: 0, Confianza: 0.34, BBox: (65.05, 131.23, 106.27, 145.52)
Clase: 0, Confianza: 0.31, BBox: (214.11, 215.75, 240.00, 239.83)
Clase: 0, Confianza: 0.31, BBox: (143.89, 205.78

In [31]:
filename = f"/content/{project}/{experiment}/results.pkl"
save_results(results, filename)

#### Test 2 (medium weed)

In [32]:
image = '/content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/503_118_29_JPG.tile01x01.rf.33b1e23ce2d5d55c753d42f7fae09523.jpg'
keyword = '/images/'
file_name = image[image.find(keyword)+len(keyword):]

In [33]:
project="model3(Epochs)"
experiment="experiment2"

In [34]:
# Ejecutar predicción sobre la imagen
results = model3(image, save=True, project=project, name=experiment)

# Acceder a las coordenadas de los bounding boxes
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Coordenadas del bounding box (xmin, ymin, xmax, ymax)
        conf = box.conf[0]  # Confianza de la predicción
        cls = box.cls[0]  # Clase predicha
        print(f"Clase: {int(cls)}, Confianza: {conf:.2f}, BBox: ({x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f})")


image 1/1 /content/drive/MyDrive/YOLO/3.5m.v3i.yolov8/valid/images/503_118_29_JPG.tile01x01.rf.33b1e23ce2d5d55c753d42f7fae09523.jpg: 640x640 12 plant-weeds, 7.7ms
Speed: 2.8ms preprocess, 7.7ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to model3(Epochs)/experiment2
Clase: 0, Confianza: 0.57, BBox: (10.41, 0.50, 54.67, 44.07)
Clase: 0, Confianza: 0.55, BBox: (0.00, 211.28, 18.89, 240.00)
Clase: 0, Confianza: 0.45, BBox: (95.27, 165.23, 107.33, 180.85)
Clase: 0, Confianza: 0.41, BBox: (31.92, 1.77, 61.25, 39.95)
Clase: 0, Confianza: 0.39, BBox: (13.64, 99.67, 32.03, 145.21)
Clase: 0, Confianza: 0.37, BBox: (0.00, 156.51, 33.05, 193.01)
Clase: 0, Confianza: 0.35, BBox: (0.00, 32.19, 28.34, 70.98)
Clase: 0, Confianza: 0.34, BBox: (0.00, 185.84, 27.55, 207.20)
Clase: 0, Confianza: 0.29, BBox: (0.00, 7.05, 12.83, 32.47)
Clase: 0, Confianza: 0.27, BBox: (0.00, 112.27, 26.12, 140.54)
Clase: 0, Confianza: 0.27, BBox: (0.00, 114.56, 19.30, 139.02)
Clase: 0, 

In [36]:
filename = f"/content/{project}/{experiment}/results.pkl"
save_results(results, filename)

### Save results

In [37]:
# Store model weights and metrics
save_on_cloud(source='/content/runs/', destination='/content/drive/MyDrive/save3/')

✅ Folder copied successfully:
   /content/runs/ 
  --> /content/drive/MyDrive/save3/


In [63]:
# Store experiments
save_on_cloud(source='/content/model3(Epochs)/', destination='/content/drive/MyDrive/save.Epochs/')

✅ Folder copied successfully:
   /content/model3(Epochs)/ 
  --> /content/drive/MyDrive/save.Epochs/


# Conclusions

| Metrics           | Experiment 1 _(Small & Medium)_ | Experiment 2 _(Small)_ | Experiment 3 _(Medium)_ |
|-----------------|--------------------------------|-----------------------|------------------------|
| **mAP50-95(B)** | 0.                          |                  |  |
| **Precision** | ~0.                           |          |           |
| **Recall** | ~0.                           |         |          |
| **Observations**|   |   |   |

1. **Experiment 1: _small & medium plant sizes_**

- **mAP50-95(B):**
- **Precision:**  
- **Recall:**  
- **Observations:**  

2. **Experiment 2: _small plant sizes_**

- **mAP50-95(B):**  
- **Precision:**  
- **Recall:**  
- **Observations:**  

3. **Experiment 3: _medium plant sizes_**

- **mAP50-95(B):**  
- **Precision:**  
- **Recall:**  
- **Observations:**  

#### Final Conclusions
-